# get a feel for the performance of various binary classifiers

The basic idea in this notebook is to get a sense for how various models perform.  The assessment is based simply on the train and test accuracy.

In another notebook the models will be evaluated with cross validation and compared with grid searching.

In [ ]:
# the usual
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")


# my utilities
from crash_utils.zip_code_and_borough_from_coords import zip_code_and_borough_from_coords
from crash_utils.fix_vehicle_names import fix_vehicle_names
from crash_utils.make_crash_features import make_crash_features
from crash_utils.basic_cleaning import basic_cleaning
from crash_utils.prepare_data_for_modelling import prepare_data_for_modelling

In [ ]:
data_path = "/Users/Mark/brainstation/capstone/nyc_bike_crash_analysis/data/"
df = pd.read_csv(data_path + "Motor_Vehicle_Collisions_-_Crashes.csv")

In [ ]:
# fill in missing zip coded and boroughs using lat/lon
df = zip_code_and_borough_from_coords(df)

In [ ]:
## clean up the VEHICLE TYPE CODE columns
df = fix_vehicle_names(df)

In [ ]:
# perform some basic data munging operations (see `crash_utils/basic_cleaning.py` for details)
df = basic_cleaning(df)

In [ ]:
df.shape

In [ ]:
#df.head()

In [ ]:
# prepare the data for modelling
# drop columns
# set up target
# run "make_crash_features.py"
# OHE the text columns
# count-vectorize the vehicles and crash factors

df = prepare_data_for_modelling(df)

In [ ]:
df.shape

In [ ]:
df.head()

# now test some models!

In [ ]:
# machine learning stuff
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample

## extract the features and targets from the big dataframe

In [ ]:
# extract target and features and then train-test-split

X = df.iloc[:,1:]
y = df.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

### fix the class imbalance with upsampling

In [ ]:
print(y_train.value_counts().sort_index())
n_min = np.sum(y_train == 0)
n_maj = np.sum(y_train == 1)

In [ ]:
# fix the class imbalance with upsampling

minority_mask = y_train == 0

X_upsampled, y_upsampled = resample(X_train.loc[minority_mask], 
                                    y_train.loc[minority_mask], 
                                    replace = True, 
                                    n_samples = n_maj)

X_train_bal = np.vstack((X_train[y_train == 1], X_upsampled))
y_train_bal = np.hstack((y_train[y_train == 1], y_upsampled))

In [ ]:
print(np.sum(y_train==0))
print(np.sum(y_train==1))
print(np.sum(y_train_bal==0))
print(np.sum(y_train_bal==1))

## run what was found to be the best classifier

In [ ]:
# run what was found to be the best classifier 2020-12-10 00:30:04 GridCVresults.pkl
# 2020-12-08, PCA(n_components=20), RandomForestClassifier(max_depth=40, n_estimators = 200, 'scaler': None}

from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=40, n_estimators=200)

my_pca = PCA(n_components=20).fit(X_train_bal)
X_train_bal_pca = my_pca.transform(X_train_bal)
X_test_pca = my_pca.transform(X_test)

rf.fit(X_train_bal_pca, y_train_bal)

rf.score(X_test_pca, y_test)

__reload the data__

In [ ]:
X = df.iloc[:,1:]
y = df.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [ ]:
# fix the class imbalance with upsampling

minority_mask = y_train == 0

X_upsampled, y_upsampled = resample(X_train.loc[minority_mask], 
                                    y_train.loc[minority_mask], 
                                    replace = True, 
                                    n_samples = n_maj)

X_train_bal = np.vstack((X_train[y_train == 1], X_upsampled))
y_train_bal = np.hstack((y_train[y_train == 1], y_upsampled))

__scale the data for those algorithms which would benefit (e.g., KNN)__

In [ ]:
# also scale the data for those algorithms which would benefit (e.g., KNN)

scaler = StandardScaler().fit(X_train_bal)

X_train_scaled = scaler.transform(X_train_bal)
X_test_scaled = scaler.transform(X_test)

## Logistic classifier

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C = 1e-6)
lr.fit(X_train_scaled, y_train_bal)

print("training set accuracy",round(lr.score(X_train_scaled, y_train_bal),3))
print("test set accuracy",round(lr.score(X_test_scaled, y_test),3))

In [ ]:
Cs = 10**np.arange(-9.,5.,1.)
test_score = []
train_score = []

for C in Cs:
    lr = LogisticRegression(C = C).fit(X_train_scaled, y_train_bal)
    train_score.append(lr.score(X_train_scaled, y_train_bal))
    test_score.append(lr.score(X_test_scaled, y_test))
    print(C,end="\r")

In [ ]:
max_ind = np.argmax(test_score)
print("max test score of", round(test_score[max_ind],3))
print("max test score at C = ",Cs[max_ind])

In [ ]:
# Visualize the result
plt.figure()
plt.plot(Cs, train_score, label='training set', marker='o')
plt.plot(Cs, test_score, label='test set', marker='o')
plt.xscale("log")
plt.xlabel('C (inverse regularization strength)')
plt.ylabel('accuracy score')
plt.title("Logistic Regression Classification: impact of regularization")
plt.axvline(Cs[max_ind],color="k")
plt.legend()
plt.show()

In [ ]:
# break out the coefficient values and feature names for text columns
coeffs = lr.coef_.reshape(-1)[253:]
features = X.columns[253:]
print(features.shape)
print(coeffs.shape)

In [ ]:
word_df = pd.DataFrame({"coeffs":coeffs, "word":features})
word_df.sort_values(by="coeffs",ascending=False,inplace=True)

In [ ]:
word_df.head(20).plot.bar(x = "word", rot=90, figsize=(14,8), fontsize=16, legend=None);
plt.xlabel("Coefficient value",size=18);
plt.ylabel("");
plt.title("Words that strongly predict injuries",size = 20);

## Decision Tree Classifier

In [ ]:
# instantiate and fit DTC

from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(max_depth=80, min_samples_leaf=100)
dtc.fit(X_train_bal, y_train_bal)


# score model on train and test sets
print(round(dtc.score(X_train_bal, y_train_bal),3))
print(round(dtc.score(X_test, y_test),3))

In [ ]:
# predict test and train
y_pred_test = dtc.predict(X_test)
y_pred_train= dtc.predict(X_train_bal)

# confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)
conf_df = pd.DataFrame(data = conf_matrix,
                       index = ['True Class 0','True Class 1'],
                       columns = ['Predicted Class 0','Predicted Class 1'])

conf_df

In [ ]:
counts_df = pd.DataFrame(y_test.value_counts().sort_index())
counts_df.rename(columns = {"outcome":"number"}, inplace=True)
counts_df["outcome"] = ["no injury","injury"]
counts_df.index.name = "encoding"
counts_df

In [ ]:
ConfusionMatrixDisplay(conf_matrix).plot()

__little mini hyperparameter search__

In [ ]:
#depths = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
depths = np.arange(1,211,10)
test_score = []
train_score = []

for depth in depths:
    dtc = DecisionTreeClassifier(max_depth = depth)
    dtc.fit(X_train_bal,y_train_bal)
    train_score.append(dtc.score(X_train_bal, y_train_bal))
    test_score.append(dtc.score(X_test, y_test))
    print(depth,end="\r")

In [ ]:
max_ind = np.argmax(test_score)
print("max test score of", round(test_score[max_ind],3))
print("max test score at max_depth of",depths[max_ind])

In [ ]:
# Visualize the result
plt.figure()
plt.plot(depths, train_score, label='training set', marker='o')
plt.plot(depths, test_score, label='test set', marker='o')
#plt.xscale("log")
plt.xlabel('max depth')
plt.ylabel('accuracy score')
plt.title("DTC: impact of max depth")
plt.axvline(depths[max_ind],color="k")
plt.legend()
plt.show()

In [ ]:
#min_leafs = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
min_leafs = np.arange(1,211,10)
test_score = []
train_score = []

for leaf in min_leafs:
    dtc = DecisionTreeClassifier(max_depth = 60, min_samples_leaf = leaf)
    dtc.fit(X_train_bal,y_train_bal)
    test_score.append(dtc.score(X_test, y_test))
    train_score.append(dtc.score(X_train_bal, y_train_bal))
    print(leaf,end="\r")

In [ ]:
max_ind = np.argmax(test_score)
print("max test score of", round(test_score[max_ind],3))
print("max test score at min_samples_leaf of",min_leafs[max_ind])

In [ ]:
# Visualize the result
plt.figure()
plt.plot(min_leafs, train_score, label='training set', marker='o')
plt.plot(min_leafs, test_score, label='test set', marker='o')
#plt.xscale("log")
plt.xlabel('min samples per leaf')
plt.ylabel('accuracy score')
plt.title("DTC: impact of min. samples per leaf")
plt.legend()
plt.show()

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# optimized RF classifier
rf = RandomForestClassifier(n_jobs = -1)
#rf = RandomForestClassifier(n_jobs = -1)
rf.fit(X_train_bal, y_train_bal)


# score model on train and test sets
print(round(rf.score(X_train_bal, y_train_bal),3))
print(round(rf.score(X_test, y_test),3))

In [ ]:
print(rf.min_samples_leaf)
print(rf.max_depth)
print(rf.max_samples)

In [ ]:
#depths = [1, 2, 5, 10, 20, 50, 100, 200, 500, 1000]
depths = np.arange(1,181,20)
test_score = []
train_score = []

for depth in depths:
    rf = RandomForestClassifier(max_depth = depth, n_estimators = 80, n_jobs = -1)
    rf.fit(X_train_bal,y_train_bal)
    train_score.append(rf.score(X_train_bal, y_train_bal))
    test_score.append(rf.score(X_test, y_test))
    print(depth,end="\r")

In [ ]:
max_ind = np.argmax(test_score)
print("max test score of", round(test_score[max_ind],3))
print("max test score at max_depth of",depths[max_ind])

In [ ]:
# Visualize the result
plt.figure()
plt.plot(depths, train_score, label='training set', marker='o')
plt.plot(depths, test_score, label='test set', marker='o')
#plt.xscale("log")
plt.xlabel('max depth')
plt.ylabel('accuracy score')
plt.title("Random Forest: impact of max depth")
plt.axvline(depths[max_ind],color="k")
plt.legend()
plt.show()

In [ ]:
estimators = np.arange(10,200,10)
test_score = []
train_score = []

for estimator in estimators:
    rf = RandomForestClassifier(max_depth = 80, n_estimators = estimator, n_jobs=-1)
    rf.fit(X_train,y_train)
    test_score.append(rf.score(X_test, y_test))
    train_score.append(rf.score(X_train_bal, y_train_bal))
    print(estimator,end="\r")

In [ ]:
max_ind = np.argmax(test_score)
print("max test score of", round(test_score[max_ind],3))
print("max test score at n_estimators of",estimators[max_ind])

In [ ]:
# Visualize the result
plt.figure()
plt.plot(estimators, train_score, label='training set', marker='o')
plt.plot(estimators, test_score, label='test set', marker='o')
plt.xlabel("number of estimators")
plt.ylabel('accuracy score')
plt.title("Random forest: impact of number of estimators")
plt.legend()
plt.show()

## Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train_bal)

# score model on train and test sets
print(round(gnb.score(X_train_scaled, y_train_bal),3))
print(round(gnb.score(X_test_scaled, y_test),3))

In [ ]:
# predict test and train
y_pred_test = gnb.predict(X_test_scaled)
y_pred_train= gnb.predict(X_train_scaled)

# confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)
conf_df = pd.DataFrame(data = conf_matrix,
                       index = ['True Class 0','True Class 1'],
                       columns = ['Predicted Class 0','Predicted Class 1'])

conf_df

## boosting

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier()
gb.fit(X_train_scaled, y_train_bal)


# score model on train and test sets
print(round(gb.score(X_train_scaled, y_train_bal),3))
print(round(gb.score(X_test_scaled, y_test),3))

In [ ]:
estimators = np.arange(10,200,20)
test_score = []
train_score = []

for estimator in estimators:
    gb = GradientBoostingClassifier(n_estimators = estimator)
    gb.fit(X_train_scaled,y_train_bal)
    test_score.append(gb.score(X_test_scaled, y_test))
    train_score.append(gb.score(X_train_scaled, y_train_bal))
    print(estimator,end="\r")

In [ ]:
max_ind = np.argmax(test_score)
print("max test score of", round(test_score[max_ind],3))
print("max test score at n_estimators of",estimators[max_ind])

In [ ]:
# Visualize the result
plt.figure()
plt.plot(estimators, train_score, label='training set', marker='o')
plt.plot(estimators, test_score, label='test set', marker='o')
plt.xlabel("number of estimators")
plt.ylabel('accuracy score')
plt.title("Gradient boosting: impact of number of estimators")
plt.legend()
plt.show()

### Adaboost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
#fitting Adaptive Boosting
ada = AdaBoostClassifier()
ada.fit(X_train_scaled, y_train_bal)

# score model on train and test sets
print(round(ada.score(X_train_scaled, y_train_bal),3))
print(round(ada.score(X_test_scaled, y_test),3))

## RidgeClassifier

In [ ]:
from sklearn.linear_model import RidgeClassifier

rc = RidgeClassifier()
rc.fit(X_train_scaled, y_train_bal)

# score model on train and test sets
print(round(rc.score(X_train_scaled, y_train_bal),3))
print(round(rc.score(X_test_scaled, y_test),3))

## K-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_jobs = -1)
knn.fit(X_train_scaled, y_train_bal)

# score model on train and test sets
print(round(knn.score(X_train_scaled, y_train_bal),3))
print(round(knn.score(X_test_scaled, y_test),3))

## MLP classifier

In [ ]:
from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(hidden_layer_sizes=(32,128,512,128,32), solver='lbfgs')

nn.fit(X_train_scaled,y_train_bal)

print(round(nn.score(X_train_scaled, y_train_bal),3))
print(round(nn.score(X_test_scaled, y_test),3))


#nn.fit(X_train_scaled,y_train)

# score model on train and test sets
# print(round(nn.score(X_train_scaled, y_train),3))
# print(round(nn.score(X_test_scaled, y_test),3))

In [ ]:
# predict test and train
y_pred_test = nn.predict(X_test_scaled)
y_pred_train= nn.predict(X_train_scaled)

# confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_test)
conf_df = pd.DataFrame(data = conf_matrix,
                       index = ['True Class 0','True Class 1','True Class 2'],
                       columns = ['Predicted Class 0','Predicted Class 1','Predicted Class 2'])

conf_df

## Tensorflow/Keras Feedforward NN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
# Create a new sequential model
model = keras.Sequential()

# Declare the hidden layers
model.add(layers.Dense(20, activation="relu"))
model.add(layers.Dense(100, activation="relu"))
#model.add(layers.Dense(500, activation="relu"))
model.add(layers.Dense(100, activation="relu"))
model.add(layers.Dense(20, activation="relu"))

# Declare the output layer
model.add(layers.Dense(3, activation="softmax"))

In [ ]:
model.compile(
    # Optimizer
    optimizer=keras.optimizers.Adam(),  
    
    # Loss function to minimize
    loss=keras.losses.SparseCategoricalCrossentropy(),
    
    # Metric used to evaluate model
    metrics=[keras.metrics.BinaryAccuracy()]
)

In [ ]:
# extract target and features and then train-test-split
X = df.iloc[:,1:]
y = df.iloc[:,0]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train, y_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

type(X_train_scaled)

In [ ]:
history = model.fit(X_train_scaled, y_train, epochs=50, verbose=1)

In [ ]:
# Evaluate the network
train_accuracy = history.history["binary_accuracy"][-1]

result = model.evaluate(X_test,y_test, verbose=0)

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {result[1]:.4f}") 